In [165]:
import numpy as np
import pandas as pd
import xlsxwriter
import time
import h5toDF
import xlautofit
import math
from summary_functions import *

In [166]:
#################################### WHERE ARE YOU RUNNING? ####################################
model_dir = 'Z:/Stefan/Soundcast_feb_twg/'


### OTHER PATHS. FOR A TYPICAL RUN, YOU DON'T HAVE TO CHANGE THESE ######################################
h5_results_file = 'outputs/daysim_outputs.h5'
h5_results_name = 'DaysimOutputs'
h5_comparison_file = 'scripts/summarize/survey.h5'
h5_comparison_name = 'Survey'
guidefile = 'scripts/summarize/inputs/calibration/CatVarDict.xlsx'
districtfile = 'scripts/summarize/inputs/calibration/TAZ_TAD_County.csv'
report_output_location = 'outputs'

parcel_decay_file = 'inputs/buffered_parcels.dat'



h5_results_file = model_dir + h5_results_file
h5_comparison_file =  model_dir + h5_comparison_file
guidefile = model_dir + guidefile
districtfile = model_dir + districtfile
report_output_location = 'outputs'


In [167]:
#READ IN YOUR DATA
data1 = h5toDF.convert(h5_results_file,guidefile,h5_results_name)
zone_district = pd.DataFrame.from_csv(districtfile, index_col = None)

---Begin DaysimOutputs conversion---
Guide import complete
Guide converted to dictionary in 0.0 seconds
Household File import/recode complete in 1.9 seconds
HouseholdDay File import/recode complete in 0.7 seconds
Person File import/recode complete in 9.9 seconds
PersonDay File import/recode complete in 4.2 seconds
Tour File import/recode complete in 9.6 seconds
Trip File import/recode complete in 22.2 seconds
---DaysimOutputs import/recode complete in 48.5 seconds---


In [168]:
trip_variables = ['otaz', 'dtaz', 'travtime', 'travcost', 'travdist', 'pno', 'mode', 'tour_id', 'opcl', 'dpcl', 'dorp']
hh_variables = ['hhno', 'hhincome', 'hhvehs', 'hhtaz']
person_variables = ['hhno', 'pno', 'pagey', 'pgend', 'id']

In [169]:
def get_variables_trips_model(output_df,trip_variables, hh_variables, person_variables):
    trip_data = output_df['Trip'][trip_variables]
    hh_data = output_df['Household'][hh_variables]
    person_data = output_df['Person'][person_variables]
    tour_data = output_df['Tour'][['hhno', 'pno', 'id']]
    tour_data.rename(columns = {'id': 'tour_id'}, inplace = True)

    merge_hh_person = pd.merge(hh_data, person_data, 'inner', on = 'hhno')
    merge_hh_person.reset_index()
    tour_data.reset_index()
    merge_hh_tour = pd.merge(merge_hh_person, tour_data, 'inner', on =('hhno', 'pno'))
    merge_trip_hh = pd.merge(merge_hh_tour, trip_data, 'outer', on= 'tour_id')
    return merge_trip_hh  

In [170]:
trips_model = get_variables_trips_model(data1, trip_variables, hh_variables, person_variables)

In [171]:
low_inc_trips = trips_model.loc[(trips_model['hhincome']<25000)]

In [172]:
low_inc_length = low_inc_trips['travdist'].mean()

In [173]:
low_inc_length

4.524602349104249

In [174]:
low_inc_driver_trips = low_inc_trips.loc[(low_inc_trips['dorp']=='Driver')]

In [175]:
low_inc_vmt = low_inc_driver_trips['travdist'].sum()

In [176]:
low_inc_vmt

4625490.466874632

In [177]:

def get_variables_persons(output_df, hh_variables, person_variables):
    
    person_variables = ['hhno', 'pno', 'pagey', 'pgend', 'pwaudist', 'pwtyp']
    hh_variables = ['hhno', 'hhincome', 'hhvehs', 'hhtaz', 'hhexpfac']
    hh_data = output_df['Household'][hh_variables]
    person_data = output_df['Person'][person_variables]

    merge_hh_person = pd.merge(hh_data, person_data, 'inner', on = 'hhno')
    merge_hh_person.reset_index()

    return merge_hh_person

In [178]:
people_model = get_variables_persons(data1, hh_variables, person_variables)

In [179]:
low_inc_people = people_model.loc[(people_model['hhincome']<25000)]

In [180]:
low_inc_ct = low_inc_people.count()['hhno']

In [181]:
low_inc_vmt/low_inc_ct

10.506198492434368

In [182]:
low_inc_wrkr = low_inc_people.loc[(low_inc_people['pwtyp']!= 'Not a Paid Worker')]

In [183]:
low_inc_wrkr['pwaudist'].mean()

8.787918742353263

In [184]:
trips_by_mode =low_inc_trips.groupby('mode').count()['hhno']

In [185]:
l_inc_modes = trips_by_mode/low_inc_trips.count()['hhno']

In [186]:
l_inc_modes.to_clipboard()